In [2]:
import numpy as np
from pygsti.baseobjs import Basis

np.set_printoptions(precision=1, linewidth=1000)

c:\Users\jkskolf\AppData\Local\miniconda3\envs\qcvv\lib\site-packages\pygsti\baseobjs\opcalc\__init__.py:31: UserWarning: 
An optimized Cython-based implementation of `pygsti.baseobjs.opcalc` is available as
an extension, but couldn't be imported. This might happen if the
extension has not been built. `pip install cython`, then reinstall
pyGSTi to build Cython extensions. Alternatively, setting the
environment variable `PYGSTI_NO_CYTHON_WARNING` will suppress this
message.

  _warnings.warn(warn_msg)
c:\Users\jkskolf\AppData\Local\miniconda3\envs\qcvv\lib\site-packages\pygsti\circuits\circuitparser\__init__.py:60: UserWarning: 
An optimized Cython-based implementation of `pygsti.circuits.circuitparser` is available as
an extension, but couldn't be imported. This might happen if the
extension has not been built. `pip install cython`, then reinstall
pyGSTi to build Cython extensions. Alternatively, setting the
environment variable `PYGSTI_NO_CYTHON_WARNING` will suppress this
message.

 

In [3]:
# 1-qubit base case
pp = Basis.cast('PP', dim=4)
pauliNames = ["I", "X", "Y", "Z"]
## HEY THIS NEEDS TO BE PER QUBIT +/-!!!
initialStates = ["X+", "X-", "Y+","Y-", "Z+", "Z-"]


In [4]:
# Commutator Helper Functions
def commute(mat1, mat2):
    return mat1@mat2 + mat2@mat1

def anti_commute(mat1, mat2):
    return mat1@mat2 - mat2@mat1

In [5]:
# Hamiltonian Error Generator
def hamiltonian_error_generator(initial_state, pauli_index, pauliDict, numQubits):
    return -1j*pauliDict[pauli_index]@initial_state@pauliDict["I"*numQubits] + 1j*pauliDict["I"*numQubits]@initial_state@pauliDict[pauli_index]

# Stochastic Error Generator
def stochastic_error_generator(initial_state, pauli_index, pauliDict, numQubits):
    return pauliDict[pauli_index]@initial_state@pauliDict[pauli_index] - pauliDict["I"*numQubits]@initial_state@pauliDict["I"*numQubits]

# Pauli-correlation Error Generator
def pauli_correlation_error_generator(initial_state, pauli_index_1, pauli_index_2, pauliDict):
    return pauliDict[pauli_index_1]@initial_state@pauliDict[pauli_index_2] + pauliDict[pauli_index_2]@initial_state@pauliDict[pauli_index_1] - 0.5 * commute(commute(pauliDict[pauli_index_1], pauliDict[pauli_index_2]), initial_state)

# Anti-symmetric Error Generator
def anti_symmetric_error_generator(initial_state, pauli_index_1, pauli_index_2, pauliDict):
    return 1j*(pauliDict[pauli_index_1]@initial_state@pauliDict[pauli_index_2] - pauliDict[pauli_index_2]@initial_state@pauliDict[pauli_index_1] + 0.5 * commute(anti_commute(pauliDict[pauli_index_1], pauliDict[pauli_index_2]), initial_state))

In [6]:
from itertools import product
# Convert basis; 1-q only at the moment
def convert_to_pauli(matrix, numQubits):
    pauliNames1Q = ["I", "X", "Y", "Z"]
    # Hard force to 1- or 2-qubit
    if numQubits == 1:
        pauliNames = pauliNames1Q
    elif numQubits == 2:
        pauliNames = [''.join(name) for name in product(pauliNames1Q, pauliNames1Q)]
    translationMatrix = pp.from_std_transform_matrix
    coefs = np.real_if_close(np.dot(translationMatrix,matrix.flatten()))
    return [(a,b) for (a,b) in zip(coefs,pauliNames) if abs(a) > 0.0001]

In [7]:
# Compute the set of measurable effects of Hamiltonian error generators operating on two qubits in each of the specified eigenstates
hamiltonianIndices = pauliNames[1:]
hamiltonianErrorOutputs = dict()
numQubits = 1
for index in hamiltonianIndices:
    for state in initialStates:
        if state[-1] == "+":
            hamiltonianErrorOutputs[(index, state)] = 0.5*hamiltonian_error_generator(pp["I"*numQubits], index, pp, numQubits) + 0.5*hamiltonian_error_generator(pp[state[:-1]], index, pp, numQubits)
        elif state[-1] == "-":
            hamiltonianErrorOutputs[(index, state)] = 0.5*hamiltonian_error_generator(pp["I"*numQubits], index, pp, numQubits) - 0.5*hamiltonian_error_generator(pp[state[:-1]], index, pp, numQubits)

# Convert measurable effects into coefficients
for key in hamiltonianErrorOutputs:
    hamiltonianErrorOutputs[key] = convert_to_pauli(hamiltonianErrorOutputs[key], numQubits)

for key in hamiltonianErrorOutputs:
    print(key, "\n", hamiltonianErrorOutputs[key])

('X', 'X+') 
 []
('X', 'X-') 
 []
('X', 'Y+') 
 [(1.0, 'Z')]
('X', 'Y-') 
 [(-1.0, 'Z')]
('X', 'Z+') 
 [(-1.0, 'Y')]
('X', 'Z-') 
 [(1.0, 'Y')]
('Y', 'X+') 
 [(-1.0, 'Z')]
('Y', 'X-') 
 [(1.0, 'Z')]
('Y', 'Y+') 
 []
('Y', 'Y-') 
 []
('Y', 'Z+') 
 [(1.0, 'X')]
('Y', 'Z-') 
 [(-1.0, 'X')]
('Z', 'X+') 
 [(1.0, 'Y')]
('Z', 'X-') 
 [(-1.0, 'Y')]
('Z', 'Y+') 
 [(-1.0, 'X')]
('Z', 'Y-') 
 [(1.0, 'X')]
('Z', 'Z+') 
 []
('Z', 'Z-') 
 []


In [8]:
# Compute the set of measurable effects of stochastic error generators operating on a single qubit in each of the specified eigenstates
stochasticIndices = ["X", "Y", "Z"]
stochasticErrorOutputs = dict()
for index in stochasticIndices:
    for state in initialStates:
        if state[-1] == "+":
            stochasticErrorOutputs[(index, state)] = 0.5*stochastic_error_generator(pp["I"], index, pp,1) + 0.5*stochastic_error_generator(pp[state[0]], index, pp,1)
        elif state[-1] == "-":
            stochasticErrorOutputs[(index, state)] = 0.5*stochastic_error_generator(pp["I"], index, pp,1) - 0.5*stochastic_error_generator(pp[state[0]], index, pp,1)

# Convert measurable effects into coefficients
for key in stochasticErrorOutputs:
    stochasticErrorOutputs[key] = convert_to_pauli(stochasticErrorOutputs[key], numQubits)
for key in stochasticErrorOutputs:
    print(key, "\n", stochasticErrorOutputs[key])


('X', 'X+') 
 []
('X', 'X-') 
 []
('X', 'Y+') 
 [(-1.0, 'Y')]
('X', 'Y-') 
 [(1.0, 'Y')]
('X', 'Z+') 
 [(-1.0, 'Z')]
('X', 'Z-') 
 [(1.0, 'Z')]
('Y', 'X+') 
 [(-1.0, 'X')]
('Y', 'X-') 
 [(1.0, 'X')]
('Y', 'Y+') 
 []
('Y', 'Y-') 
 []
('Y', 'Z+') 
 [(-1.0, 'Z')]
('Y', 'Z-') 
 [(1.0, 'Z')]
('Z', 'X+') 
 [(-1.0, 'X')]
('Z', 'X-') 
 [(1.0, 'X')]
('Z', 'Y+') 
 [(-1.0, 'Y')]
('Z', 'Y-') 
 [(1.0, 'Y')]
('Z', 'Z+') 
 []
('Z', 'Z-') 
 []


In [9]:
# Compute the set of measurable effects of pauli-correlation error generators operating on a single qubit in each of the specified eigenstates
pauliCorrelationIndices = ["XY", "XZ", "YZ"]
pauliCorrelationErrorOutputs = dict()
for index in pauliCorrelationIndices:
    for state in initialStates:
        if state[-1] == "+":
            pauliCorrelationErrorOutputs[(index, state)] = 0.5*pauli_correlation_error_generator(pp["I"], index[0], index[1], pp) + 0.5*pauli_correlation_error_generator(pp[state[0]], index[0], index[1], pp)
        elif state[-1] == "-":
            pauliCorrelationErrorOutputs[(index, state)] = 0.5*pauli_correlation_error_generator(pp["I"], index[0], index[1], pp) - 0.5*pauli_correlation_error_generator(pp[state[0]], index[0], index[1], pp)

# Convert measurable effects into coefficients
for key in pauliCorrelationErrorOutputs:
    pauliCorrelationErrorOutputs[key] = convert_to_pauli(pauliCorrelationErrorOutputs[key], numQubits)
for key in pauliCorrelationErrorOutputs:
    print(key, "\n", pauliCorrelationErrorOutputs[key])


('XY', 'X+') 
 [(1.0, 'Y')]
('XY', 'X-') 
 [(-1.0, 'Y')]
('XY', 'Y+') 
 [(1.0, 'X')]
('XY', 'Y-') 
 [(-1.0, 'X')]
('XY', 'Z+') 
 []
('XY', 'Z-') 
 []
('XZ', 'X+') 
 [(1.0, 'Z')]
('XZ', 'X-') 
 [(-1.0, 'Z')]
('XZ', 'Y+') 
 []
('XZ', 'Y-') 
 []
('XZ', 'Z+') 
 [(1.0, 'X')]
('XZ', 'Z-') 
 [(-1.0, 'X')]
('YZ', 'X+') 
 []
('YZ', 'X-') 
 []
('YZ', 'Y+') 
 [(1.0, 'Z')]
('YZ', 'Y-') 
 [(-1.0, 'Z')]
('YZ', 'Z+') 
 [(1.0, 'Y')]
('YZ', 'Z-') 
 [(-1.0, 'Y')]


In [10]:
# Compute the set of measurable effects of pauli-correlation error generators operating on a single qubit in each of the specified eigenstates
antiSymmetricIndices = ["XY", "XZ", "YZ"]
antiSymmetricErrorOutputs = dict()
for index in antiSymmetricIndices:
    for state in initialStates:
        if state[-1] == "+":
            antiSymmetricErrorOutputs[(index, state)] = 0.5*anti_symmetric_error_generator(pp["I"], index[0], index[1], pp) + 0.5*anti_symmetric_error_generator(pp[state[0]], index[0], index[1], pp)
        elif state[-1] == "-":
            antiSymmetricErrorOutputs[(index, state)] = 0.5*anti_symmetric_error_generator(pp["I"], index[0], index[1], pp) - 0.5*anti_symmetric_error_generator(pp[state[0]], index[0], index[1], pp)

# Convert measurable effects into coefficients
for key in antiSymmetricErrorOutputs:
    antiSymmetricErrorOutputs[key] = convert_to_pauli(antiSymmetricErrorOutputs[key], numQubits)
for key in antiSymmetricErrorOutputs:
    print(key, "\n", antiSymmetricErrorOutputs[key])


('XY', 'X+') 
 [(-2.0, 'Z')]
('XY', 'X-') 
 [(-2.0, 'Z')]
('XY', 'Y+') 
 [(-2.0, 'Z')]
('XY', 'Y-') 
 [(-2.0, 'Z')]
('XY', 'Z+') 
 [(-2.0, 'Z')]
('XY', 'Z-') 
 [(-2.0, 'Z')]
('XZ', 'X+') 
 [(2.0, 'Y')]
('XZ', 'X-') 
 [(2.0, 'Y')]
('XZ', 'Y+') 
 [(2.0, 'Y')]
('XZ', 'Y-') 
 [(2.0, 'Y')]
('XZ', 'Z+') 
 [(2.0, 'Y')]
('XZ', 'Z-') 
 [(2.0, 'Y')]
('YZ', 'X+') 
 [(-2.0, 'X')]
('YZ', 'X-') 
 [(-2.0, 'X')]
('YZ', 'Y+') 
 [(-2.0, 'X')]
('YZ', 'Y-') 
 [(-2.0, 'X')]
('YZ', 'Z+') 
 [(-2.0, 'X')]
('YZ', 'Z-') 
 [(-2.0, 'X')]


In [11]:
# 2-Qubit case
from itertools import product
pp = Basis.cast('PP', dim=16)
pauliNames1Q = ["I", "X", "Y", "Z"]
pauliNames = [''.join(name) for name in product(pauliNames1Q, pauliNames1Q)]
initialStates = [''.join((name, '+')) for name in pauliNames[1:]] + [''.join((name, '-')) for name in pauliNames[1:]]

In [12]:
print(pauliNames)
print(initialStates)

['II', 'IX', 'IY', 'IZ', 'XI', 'XX', 'XY', 'XZ', 'YI', 'YX', 'YY', 'YZ', 'ZI', 'ZX', 'ZY', 'ZZ']
['IX+', 'IY+', 'IZ+', 'XI+', 'XX+', 'XY+', 'XZ+', 'YI+', 'YX+', 'YY+', 'YZ+', 'ZI+', 'ZX+', 'ZY+', 'ZZ+', 'IX-', 'IY-', 'IZ-', 'XI-', 'XX-', 'XY-', 'XZ-', 'YI-', 'YX-', 'YY-', 'YZ-', 'ZI-', 'ZX-', 'ZY-', 'ZZ-']


In [13]:
print(pp)
pp1Q = Basis.cast('PP', dim = 4)
print(pp1Q)
print(pp1Q["X"])
print(np.tensordot(pp1Q["X"], pp1Q["X"], 0))

Pauli-Product basis (dim=16), 16 elements of shape (4, 4) :
II, IX, IY, IZ, XI, XX, XY, XZ, YI, YX, YY, YZ, ZI, ZX, ZY, ZZ
Pauli-Product basis (dim=4), 4 elements of shape (2, 2) :
I, X, Y, Z
[[0.+0.j 1.+0.j]
 [1.+0.j 0.+0.j]]
[[[[0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j]]

  [[0.+0.j 1.+0.j]
   [1.+0.j 0.+0.j]]]


 [[[0.+0.j 1.+0.j]
   [1.+0.j 0.+0.j]]

  [[0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j]]]]


In [14]:
# Compute the set of measurable effects of Hamiltonian error generators operating on two qubits in each of the specified eigenstates
hamiltonianIndices = pauliNames[1:]
hamiltonianErrorOutputs = dict()
numQubits = 2
for index in hamiltonianIndices:
    for state in initialStates:
        if state[-1] == "+":
            hamiltonianErrorOutputs[(index, state)] = 0.5*hamiltonian_error_generator(pp["I"*numQubits], index, pp, numQubits) + 0.5*hamiltonian_error_generator(pp[state[:-1]], index, pp, numQubits)
        elif state[-1] == "-":
            hamiltonianErrorOutputs[(index, state)] = 0.5*hamiltonian_error_generator(pp["I"*numQubits], index, pp, numQubits) - 0.5*hamiltonian_error_generator(pp[state[:-1]], index, pp, numQubits)

# Convert measurable effects into coefficients
for key in hamiltonianErrorOutputs:
    hamiltonianErrorOutputs[key] = convert_to_pauli(hamiltonianErrorOutputs[key], numQubits)

for key in hamiltonianErrorOutputs:
    print(key, "\n", hamiltonianErrorOutputs[key])

('IX', 'IX+') 
 []
('IX', 'IY+') 
 [(1.0, 'IZ')]
('IX', 'IZ+') 
 [(-1.0, 'IY')]
('IX', 'XI+') 
 []
('IX', 'XX+') 
 []
('IX', 'XY+') 
 [(1.0, 'XZ')]
('IX', 'XZ+') 
 [(-1.0, 'XY')]
('IX', 'YI+') 
 []
('IX', 'YX+') 
 []
('IX', 'YY+') 
 [(1.0, 'YZ')]
('IX', 'YZ+') 
 [(-1.0, 'YY')]
('IX', 'ZI+') 
 []
('IX', 'ZX+') 
 []
('IX', 'ZY+') 
 [(1.0, 'ZZ')]
('IX', 'ZZ+') 
 [(-1.0, 'ZY')]
('IX', 'IX-') 
 []
('IX', 'IY-') 
 [(-1.0, 'IZ')]
('IX', 'IZ-') 
 [(1.0, 'IY')]
('IX', 'XI-') 
 []
('IX', 'XX-') 
 []
('IX', 'XY-') 
 [(-1.0, 'XZ')]
('IX', 'XZ-') 
 [(1.0, 'XY')]
('IX', 'YI-') 
 []
('IX', 'YX-') 
 []
('IX', 'YY-') 
 [(-1.0, 'YZ')]
('IX', 'YZ-') 
 [(1.0, 'YY')]
('IX', 'ZI-') 
 []
('IX', 'ZX-') 
 []
('IX', 'ZY-') 
 [(-1.0, 'ZZ')]
('IX', 'ZZ-') 
 [(1.0, 'ZY')]
('IY', 'IX+') 
 [(-1.0, 'IZ')]
('IY', 'IY+') 
 []
('IY', 'IZ+') 
 [(1.0, 'IX')]
('IY', 'XI+') 
 []
('IY', 'XX+') 
 [(-1.0, 'XZ')]
('IY', 'XY+') 
 []
('IY', 'XZ+') 
 [(1.0, 'XX')]
('IY', 'YI+') 
 []
('IY', 'YX+') 
 [(-1.0, 'YZ')]
('IY', 'YY+') 
 [

In [15]:
import pygsti
from pygsti.extras import idletomography as idt

n_qubits = 1
gates = ["Gi","Gx","Gy","Gcnot"]
max_lengths = [1,2,4,8]
pspec = pygsti.processors.QubitProcessorSpec(n_qubits, gates, geometry='line', nonstd_gate_unitaries={():1})

mdl_target = pygsti.models.create_crosstalk_free_model(pspec)
paulidicts = idt.determine_paulidicts(mdl_target)
idle_experiments = idt.make_idle_tomography_list(n_qubits, max_lengths, paulidicts, maxweight=1)
print(len(idle_experiments), "idle tomography experiments for %d qubits" % n_qubits)

36 idle tomography experiments for 1 qubits


In [16]:
mdl_target.operation_blks['layers'].keys()

odict_keys([Label(('Gi', 0)), Label(('Gx', 0)), Label(('Gy', 0))])

In [17]:
mdl_target.probabilities(pygsti.circuits.Circuit([[]],line_labels=(0,)))

OutcomeLabelDict([(('0',), 1.0), (('1',), 1.0146536357569526e-17)])

In [18]:
print(mdl_target)

layers:rho0 = Computational Z-basis state vec for 1 qubits w/z-values: [0]

layers:Mdefault = Computational(Z)-basis POVM on 1 qubits and filter None


gates:Gi = 
StaticStandardOp with name Gi and evotype densitymx_slow


gates:Gx = 
StaticStandardOp with name Gx and evotype densitymx_slow


gates:Gy = 
StaticStandardOp with name Gy and evotype densitymx_slow


gates:Gcnot = 
StaticStandardOp with name Gcnot and evotype densitymx_slow


layers:Gi:0 = 
StaticStandardOp with name Gi and evotype densitymx_slow


layers:Gx:0 = 
StaticStandardOp with name Gx and evotype densitymx_slow


layers:Gy:0 = 
StaticStandardOp with name Gy and evotype densitymx_slow







In [19]:
from pygsti.baseobjs import Label
updated_ckt_list= []
for ckt in idle_experiments:
    new_ckt= ckt.copy(editable=True)
    for i,lbl in enumerate(ckt):
        if lbl == Label(()):
            new_ckt[i] = Label(('Gi',0))
    updated_ckt_list.append(new_ckt)

In [20]:
#Generate some data (takes ~5min w/10Q)
from pygsti.baseobjs import Label 
mdl_datagen = pygsti.models.create_crosstalk_free_model(pspec,
                                                        lindblad_error_coeffs={'Gi': {'HX': 0.01, 'SX': 0.01}})
# Error models! Random with right CP constraints from Taxonomy paper
ds = pygsti.data.simulate_data(mdl_datagen, updated_ckt_list, 100000, seed=8675309)

c:\Users\jkskolf\AppData\Local\miniconda3\envs\qcvv\lib\site-packages\pygsti\circuits\circuit.py:816: UserWarning: Editable circuit is being converted to read-only mode in order to hash it.  You should call circuit.done_editing() beforehand.
  _warnings.warn(("Editable circuit is being converted to read-only"


In [21]:
print(idle_experiments)

[Circuit(Gy:0[]Gy:0@(0)), Circuit(Gy:0[][]Gy:0@(0)), Circuit(Gy:0[][][][]Gy:0@(0)), Circuit(Gy:0[][][][][][][][]Gy:0@(0)), Circuit(Gx:0[]Gx:0@(0)), Circuit(Gx:0[][]Gx:0@(0)), Circuit(Gx:0[][][][]Gx:0@(0)), Circuit(Gx:0[][][][][][][][]Gx:0@(0)), Circuit([]@(0)), Circuit([][]@(0)), Circuit([][][][]@(0)), Circuit([][][][][][][][]@(0)), Circuit(Gy:0Gy:0Gy:0[]Gy:0Gy:0Gy:0@(0)), Circuit(Gy:0Gy:0Gy:0[][]Gy:0Gy:0Gy:0@(0)), Circuit(Gy:0Gy:0Gy:0[][][][]Gy:0Gy:0Gy:0@(0)), Circuit(Gy:0Gy:0Gy:0[][][][][][][][]Gy:0Gy:0Gy:0@(0)), Circuit(Gx:0Gx:0Gx:0[]Gx:0Gx:0Gx:0@(0)), Circuit(Gx:0Gx:0Gx:0[][]Gx:0Gx:0Gx:0@(0)), Circuit(Gx:0Gx:0Gx:0[][][][]Gx:0Gx:0Gx:0@(0)), Circuit(Gx:0Gx:0Gx:0[][][][][][][][]Gx:0Gx:0Gx:0@(0)), Circuit(Gx:0Gx:0[]Gx:0Gx:0@(0)), Circuit(Gx:0Gx:0[][]Gx:0Gx:0@(0)), Circuit(Gx:0Gx:0[][][][]Gx:0Gx:0@(0)), Circuit(Gx:0Gx:0[][][][][][][][]Gx:0Gx:0@(0)), Circuit([]Gx:0@(0)), Circuit([][]Gx:0@(0)), Circuit([][][][]Gx:0@(0)), Circuit([][][][][][][][]Gx:0@(0)), Circuit(Gy:0[]@(0)), Circuit(Gy:0

In [22]:
idle_experiments[0][1]

Label(())

In [23]:
print(ds)

Gy:0Gi:0Gy:0@(0)  :  {('0',): 0.0, ('1',): 100000.0}
Gy:0Gi:0Gi:0Gy:0@(0)  :  {('0',): 0.0, ('1',): 100000.0}
Gy:0Gi:0Gi:0Gi:0Gi:0Gy:0@(0)  :  {('0',): 0.0, ('1',): 100000.0}
Gy:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gy:0@(0)  :  {('0',): 0.0, ('1',): 100000.0}
Gx:0Gi:0Gx:0@(0)  :  {('0',): 524.0, ('1',): 99476.0}
Gx:0Gi:0Gi:0Gx:0@(0)  :  {('0',): 1001.0, ('1',): 98999.0}
Gx:0Gi:0Gi:0Gi:0Gi:0Gx:0@(0)  :  {('0',): 1977.0, ('1',): 98023.0}
Gx:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gx:0@(0)  :  {('0',): 4124.0, ('1',): 95876.0}
Gi:0@(0)  :  {('0',): 99501.0, ('1',): 499.0}
Gi:0Gi:0@(0)  :  {('0',): 98939.0, ('1',): 1061.0}
Gi:0Gi:0Gi:0Gi:0@(0)  :  {('0',): 98007.0, ('1',): 1993.0}
Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0@(0)  :  {('0',): 95854.0, ('1',): 4146.0}
Gy:0Gy:0Gy:0Gi:0Gy:0Gy:0Gy:0@(0)  :  {('0',): 0.0, ('1',): 100000.0}
Gy:0Gy:0Gy:0Gi:0Gi:0Gy:0Gy:0Gy:0@(0)  :  {('0',): 0.0, ('1',): 100000.0}
Gy:0Gy:0Gy:0Gi:0Gi:0Gi:0Gi:0Gy:0Gy:0Gy:0@(0)  :  {('0',): 0.0, ('1',): 100000.0}
Gy:0Gy:0Gy:0Gi:0Gi:0Gi:0Gi:

In [24]:
#Run idle tomography (takes ~1.5min w/10Q)
results = idt.do_idle_tomography(n_qubits, ds, max_lengths, paulidicts, maxweight=1, advanced_options={"jacobian mode":"together"}, idle_string='Gi:0')

In [25]:
print(results.observed_rate_infos.keys())

dict_keys(['samebasis', 'diffbasis'])


In [26]:
print(len(results.observed_rate_infos['samebasis']))

6


In [27]:
print(len(results.pauli_fidpairs['samebasis']))

6


In [28]:
results.pauli_fidpairs['samebasis']

[(State[+X], State[-X]),
 (State[-Y], State[+Y]),
 (State[+Z], State[+Z]),
 (State[-X], State[+X]),
 (State[+Y], State[-Y]),
 (State[-Z], State[-Z])]

In [29]:
results.pauli_fidpairs['diffbasis']

[(State[+Z], State[+Y]), (State[+X], State[+Z]), (State[-Y], State[-X])]

In [40]:
results.observed_rate_infos['diffbasis']

[OrderedDict([(NQPauliOp[ Y],
               {'rate': -0.01249839683278231,
                'fit_order': 1,
                'fitCoeffs': array([-0., -0.]),
                'data': [-0.00998, -0.02726, -0.05346, -0.0991],
                'errbars': [0.003162120174186933,
                 0.003161102485526213,
                 0.003157755576988187,
                 0.003146711283228889],
                'weights': [632.4869664443099,
                 632.6904723679822,
                 633.3608680201303,
                 635.5834462698276],
                'jacobian row': array([-1.,  0.,  0.]),
                'affine jacobian row': array([0, 2, 0])})]),
 OrderedDict([(NQPauliOp[ Z],
               {'rate': 9.791318297408492e-05,
                'fit_order': 1,
                'fitCoeffs': array([ 9.8e-05, -2.7e-03]),
                'data': [-0.00298, -0.00312, -0.00086, -0.0025],
                'errbars': [0.0031622636189919395,
                 0.003162262268693095,
                

In [42]:
results.observed_rate_infos['samebasis'][0].keys()

TypeError: 'odict_keys' object is not subscriptable

In [32]:
results.error_list

[NQPauliOp[ X], NQPauliOp[ Y], NQPauliOp[ Z]]

In [33]:
ws = pygsti.report.Workspace()
#ws.init_notebook_mode(autodisplay=True)

In [34]:
print(results)

Idle Tomography Results
Intrinsic stochastic rates: 
   X: 0.0051644
   Y: 9.01528e-06
   Z: -9.01528e-06
Intrinsic affine rates: 
   X: 8.15213e-21
   Y: -7.28703e-05
   Z: 1.79652e-05
Intrinsic hamiltonian rates:
   X: 0.0123527
   Y: -6.19827e-05
   Z: -0.000641576



In [35]:
print(ds)

Gy:0Gi:0Gy:0@(0)  :  {('0',): 0.0, ('1',): 100000.0}
Gy:0Gi:0Gi:0Gy:0@(0)  :  {('0',): 0.0, ('1',): 100000.0}
Gy:0Gi:0Gi:0Gi:0Gi:0Gy:0@(0)  :  {('0',): 0.0, ('1',): 100000.0}
Gy:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gy:0@(0)  :  {('0',): 0.0, ('1',): 100000.0}
Gx:0Gi:0Gx:0@(0)  :  {('0',): 524.0, ('1',): 99476.0}
Gx:0Gi:0Gi:0Gx:0@(0)  :  {('0',): 1001.0, ('1',): 98999.0}
Gx:0Gi:0Gi:0Gi:0Gi:0Gx:0@(0)  :  {('0',): 1977.0, ('1',): 98023.0}
Gx:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gx:0@(0)  :  {('0',): 4124.0, ('1',): 95876.0}
Gi:0@(0)  :  {('0',): 99501.0, ('1',): 499.0}
Gi:0Gi:0@(0)  :  {('0',): 98939.0, ('1',): 1061.0}
Gi:0Gi:0Gi:0Gi:0@(0)  :  {('0',): 98007.0, ('1',): 1993.0}
Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0@(0)  :  {('0',): 95854.0, ('1',): 4146.0}
Gy:0Gy:0Gy:0Gi:0Gy:0Gy:0Gy:0@(0)  :  {('0',): 0.0, ('1',): 100000.0}
Gy:0Gy:0Gy:0Gi:0Gi:0Gy:0Gy:0Gy:0@(0)  :  {('0',): 0.0, ('1',): 100000.0}
Gy:0Gy:0Gy:0Gi:0Gi:0Gi:0Gi:0Gy:0Gy:0Gy:0@(0)  :  {('0',): 0.0, ('1',): 100000.0}
Gy:0Gy:0Gy:0Gi:0Gi:0Gi:0Gi: